#### Convert Delta format and rename columns to snake_case

In [1]:
table_name = [] 

for i in dbutils.fs.ls("/mnt/silver/testdb/"):
    table_name.append(i.name.split("/")[0])                          
                                                                                    
from pyspark.sql.functions import col

# Loop through each table name
for table_name in table_name:
    try:
        # Construct the path to the Delta file in the silver mount
        silver_path = f'/mnt/silver/testdb/{table_name}'
                                    
        # Load the Delta file into a DataFrame
        df = spark.read.format('delta').load(silver_path)
                                    
        # Get the list of column names
        column_names = df.columns
                                    
        # Rename columns to snake_case
        for old_col_name in column_names:
            # Convert column name from CamelCase to snake_case
            new_col_name = "".join(["_" + char.lower() if char.isupper() and not (i > 0 and old_col_name[i-1].isupper()) else char.lower() for i, char in enumerate(old_col_name)]).lstrip("_")
                                        
            # Rename the column
            df = df.withColumnRenamed(old_col_name, new_col_name)
                                    
        # Construct the output path for the gold mount
        gold_path = f'/mnt/gold/testdb/{table_name}/'
                                    
        # Write the DataFrame to Delta format in the gold mount
        df.write.format('delta').mode('overwrite').save(gold_path)
                                    
        # Display the DataFrame 
        display(df)
                                    
        print(f"Successfully processed table: {table_name}")

    except Exception as e:
        print(f"Failed to process table: {table_name}. Error: {e}")

StatementMeta(, 94782f8d-8e1a-443b-8243-a519f11c1503, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8c8340a4-1aaf-4a8b-876b-ba53882e012c)